<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Mistral_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install mistralai

In [16]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mistralai import Mistral

# Initialize Mistral API client
api_key = "yGnufRVsYoibAfyXEopns68J4TacNiEH"  # Replace with your actual API key
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [17]:


# Define function to get Mistral AI's output for depression detection
def get_chat_completion(prompt):
    response = client.chat.complete(
        model=model,
        messages=[
            {"role": "system", "content": "Classify the following Bangla text as 'depressed' or 'not depressed'.Give no explanation. Respond with only 'depressed' or 'not depressed'."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    return response.choices[0].message.content.strip()

# Load dataset

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load and shuffle dataset
data = pd.read_excel("Bangla2.xlsx")  # Path to your dataset file
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate data by labels
depressed = data[data["labels"] == 1]
not_depressed = data[data["labels"] == 0]

# Balance the classes by taking an equal number of samples
min_samples = min(len(depressed), len(not_depressed))
depressed_sample = depressed.sample(min_samples, random_state=42)
not_depressed_sample = not_depressed.sample(min_samples, random_state=42)

# Combine the balanced samples
balanced_data = pd.concat([depressed_sample, not_depressed_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

# Extract sentences and labels
sentences = balanced_data["tweets"]  # Text column
labels = balanced_data["labels"]  # Label column

# Split data: 20% for testing
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2
)


In [21]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Generate predictions with progress bar
predictions = []
for sentence in test_sentences:
    prediction = get_chat_completion(sentence)
    # Convert "depressed"/"not depressed" response to binary labels (1 and 0)
    predictions.append(1 if prediction == "depressed" else 0)
    time.sleep(5)  # Wait for 5 seconds between API requests to respect rate limits

# Calculate metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

# Output results with 4 decimal places
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.6701
Precision: 0.7240
Recall: 0.6435
F1 Score: 0.6814
